In [46]:
# %env PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
# %env  PATH_FILE_PRIVATE_KEY_ZEROMQ_SERVER=/home/adam/Engineering/Docker/smart_broker/private_keys/server.key_secret
# %env PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys
# %env PATH_DIR_DATASET=/home/adam/datascience/data

In [47]:
class Session():
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port

session = Session("127.0.0.1", "8000")

# 1. Data Model Creation

In [48]:
# from smart_broker_api import data_model_api

# series_name = "bmi_mean"
# measurement_source_name = "Observation:Body Mass Index"
# type_aggregator = "AgregatorIntervalMean"
# unit = "kg/m2"

# data_model_api.create_series(session, series_name, measurement_source_name, type_aggregator, unit)

In [49]:
from smart_broker_api import data_model_api

data_model_id = data_model_api.create_date_frame(session, "data_frame_0")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_mean", "Observation:Body Mass Index", "AgregatorIntervalMean")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_first", "Observation:Body Mass Index", "AgregatorIntervalFirstOccurance")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_last", "Observation:Body Mass Index", "AgregatorIntervalLastOccurance")
data_model_tabular_id = data_model_api.create_tabular_data(session)
data_model_tabular_id = data_model_api.tabular_add_dataframe(session, data_model_id, data_model_tabular_id)

# 2. Parse Data from Data Model Specification

In [50]:
from smart_broker_api import data_api

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

longitudinal_id = data_api.read_longitudinal_fhirv1(session)
tabular_dataset_id = data_api.parse_dataset_tabular_from_longitudinal(session, longitudinal_id, dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_api.data_frame_tabular_select_data_frame(session, tabular_dataset_id, "data_frame_0")

In [51]:
from smart_broker_api import preprocessing_api

no_nan_data_frame_id = preprocessing_api.drop_na_data_frame(session, data_frame_id)

# 3. Grab series to work with in Statistics Lib

In [52]:
series_1_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_mean")
series_2_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

b6abe1c1-2327-4581-913b-b94bef604559 , 53d02db4-8068-451a-a4f9-7a8ab5f2bcb6


# 4. Run Statistics Library

In [53]:
from smart_broker_api import statistics_api

type_distribution="normalunit" 
type_ranking="cdf"
alternative = "two-sided"
print(statistics_api.count(session,  series_1_id))
print(statistics_api.mean(session,  series_1_id))
print(statistics_api.chisquare(session,  series_1_id, series_2_id))
print(statistics_api.kolmogorovSmirnovTest(session,  series_1_id, type_distribution, type_ranking))
print(statistics_api.kurtosis(session,  series_1_id))
print(statistics_api.levene_test(session,  series_1_id, series_2_id))
print(statistics_api.mann_whitney_u_test(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_api.min_max(session,  series_1_id))
print(statistics_api.paired_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.pearson(session,  series_1_id, series_2_id, alternative))
print(statistics_api.skewness(session,  series_1_id))
print(statistics_api.spearman(session,  series_1_id, series_2_id, alternative, type_ranking)) 
print(statistics_api.student_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.variance(session,  series_1_id))
print(statistics_api.welch_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking))







20
22.76433814224586
[380.0, 0.23583148842099294]
[0.95, 1.9073486328125338e-26]
-1.225866948308427
(0.003760306204864892, 0.9514246952891092)
(180.0, 0.5978625674290079)
(11.76951154694196, 30.172100539128525)
(-3.842534659324879, 0.00027445641167562454)
(0.9332515128886971, 1.9573960230445664e-09)
-0.08041628194553505
(0.9505828978811472, 1.394346860195128e-10)
(-0.999458438479299, 0.08097370675497725)
28.617281263722433
(-0.9994584384792992, 0.08097675209806848)
(0.9505828978811472, 1.394346860195128e-10)
